<a href="https://colab.research.google.com/github/Dntfreitas/introduction-agents-ai/blob/main/6_agents_orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents Orchestration

This notebook is a simple example of how to use the OpenAI Agents SDK to orchestrate a multi-step process.

It demonstrates how to use agents to plan, search, write a report, and send an email.

The example is a simple equity research process, where the agent plans searches, performs them, writes a report, and sends an email with the report.

In [ ]:
# Let's make sure we have the required libraries installed for this tutorial.
!pip install openai sendgrid openai-agents

In [ ]:
# Now, let's import the necessary libraries and set up our environment.
import os

import asyncio
from typing import List

from agents import Agent, Runner, WebSearchTool, ModelSettings, trace
from agents import function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from openai import OpenAI
from pydantic import BaseModel
from sendgrid import sendgrid, Email, To, Content, Mail
from IPython.display import Markdown, display

In [ ]:
# As we are going to use Google Coolab, we don't need to load the environment variables.
# Otherwise, you can use the following code to load the environment variables from a `.env` file.
# from dotenv import load_dotenv
# load_dotenv(override=True)

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
TWILIO_API_KEY = userdata.get('TWILIO_API_KEY')

In [ ]:
# Set the enviorment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
HOW_MANY_SEARCHES = 3

# Instructions

In [ ]:
PLANNER_INSTRUCTIONS = (
    "You are an equity-research strategist. Given an investment question or theme, "
    f"suggest {HOW_MANY_SEARCHES} focused stock-market searches (named tickers, "
    "sectors, or simple screening criteria) that help identify attractive companies."
)

SEARCHER_INSTRUCTIONS = (
    "You are a research assistant. Use **WebSearchTool** to pull up-to-date information "
    "for the given search term. Summarise the key take-aways in ≤ 300 words, using "
    "plain English. Highlight:\n"
    "• What the company (or sector) does\n"
    "• Latest price and 52-week range (if easily found)\n"
    "• Any notable recent news or catalysts\n"
    "Explain jargon briefly so a newcomer can follow."
)

WRITER_INSTRUCTIONS = (
    "You are a senior equity analyst writing a beginner-friendly research note. "
    "Begin with an outline, then craft a detailed markdown report (≥ 1 000 words) that covers:\n"
    "• Company/sector overview (plain English)\n"
    "• Why it might be a good investment now\n"
    "• Key numbers (define any finance terms!)\n"
    "• Risks to monitor\n"
    "Finish with 2–3 follow-up questions."
)

# Data models

In [ ]:
class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."
    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem]
    """A list of web searches to perform to best answer the query."""


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""
    markdown_report: str
    """The final report"""
    follow_up_questions: List[str]
    """Suggested topics to research further"""

# Tools

In [ ]:
@function_tool
def send_email(body: str, subject: str) -> dict:
    """
    Send out an email with the given subject and HTML body.
    :param body: the body of the email in HTML format
    :param subject: the subject of the email
    :return: the status of the email
    """
    sg = sendgrid.SendGridAPIClient(api_key=TWILIO_API_KEY)
    from_email = Email("diogo.nt.freitas@gmail.com")
    to_email = To("diogo.nt.freitas@gmail.com")
    content = Content("text/html", body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": response.status_code}

# Agents

In [ ]:
planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

search_agent = Agent(
    name="SearchAgent",
    instructions=SEARCHER_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)


In [ ]:
# HTML email body converter (this is a tool/agent)
html_instructions = f"""
{RECOMMENDED_PROMPT_PREFIX}
You can convert a text email body to an HTML email body.
You are given a text email body which might have some markdown
and you need to convert it to an HTML email body with simple, clear, compelling layout and design. Please reply only with the HTML code."""

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",
                                   tool_description="Convert a text email body to an HTML email body")

# Subject writer (this is a tool/agent)
subject_instructions = f"""
{RECOMMENDED_PROMPT_PREFIX}
You can write a subject for an email.
You are given a message and you need to write a subject for an email that is likely to get a response."""

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer",
                                      tool_description="Write a subject for a reservation email")

email_tools = [subject_tool, html_tool, send_email]

instructions = """
{RECOMMENDED_PROMPT_PREFIX}
You are an email formatter and sender. You receive the body of an email to be sent.
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML.
Finally, you use the send_email tool to send the email with the subject and HTML body."""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

# Pipeline helpers

In [ ]:
async def plan_searches(query: str) -> WebSearchPlan:
    print("Planning searches...")
    res = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(res.final_output.searches)} searches")
    return res.final_output


async def run_search(item: WebSearchItem):
    input = f"Search term: {item.query}\nReason: {item.reason}"
    res = await Runner.run(search_agent, input)
    return res.final_output


async def perform_searches(plan: WebSearchPlan):
    print("Searching...")
    tasks = [asyncio.create_task(run_search(it)) for it in plan.searches]
    res = await asyncio.gather(*tasks)
    print("Finished searching")
    return res


async def write_report(query: str, summaries: List[str]) -> ReportData:
    print("Thinking about report...")
    prompt = f"Original query: {query}\nSummaries: {summaries}"
    res = await Runner.run(writer_agent, prompt)
    print("Finished writing report")
    return res.final_output


async def send_email(report: ReportData):
    print("Writing email...")
    res = await Runner.run(emailer_agent, report.markdown_report)
    print("Email sent")
    return res


# Orchestration

In [ ]:
query = "Cost-efficient renewable-energy stocks for long-term growth"

with trace("Equity-Research"):
    plan = await plan_searches(query)
    summaries = await perform_searches(plan)
    report = await write_report(query, summaries)
    await send_email(report)
    display(Markdown(report.short_summary))
    display(Markdown(report.markdown_report))
    print("\nFollow-ups:")
    for q in report.follow_up_questions:
        print("•", q)